In [1]:
# Model-1-V2

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('https://zenodo.org/record/400614/files/apache.csv?download=1',sep=',')

In [4]:
df.head()

,id,product,component,reporter,bug_status,resolution,priority,bug_severity,version,short_desc,opendate,dup_list,root_id,disc_id
0,2,Log4j -,Layout,bugzilla,CLOS,FIXE,P3,nor,unspe,Just testing the Boogzeela setup for log4j,2001-01-08,NaN,NaN,NaN
1,3,Log4j -,Appender,matthew_scully,RESO,FIXE,P3,nor,unspe,file sharing,2001-01-08,NaN,NaN,NaN
2,22,Apache h,All,greenrd,CLOS,WONT,P5,enh,2.0-H,Need hooks for user-defined error handling,2001-01-09,NaN,NaN,NaN
3,27,Log4j -,Layout,richard.mccarthy,RESO,INVA,P1,nor,1.0,Error in the formatting of the (%F:%L) layout ...,2001-01-10,NaN,NaN,NaN
4,29,Log4j -,Other,woge,RESO,FIXE,P3,nor,1.0,"Calling Category.error(Object, Throwable) thro...",2001-01-11,NaN,NaN,NaN


**Remove NaN from the short_desc**

In [5]:
df.isnull().sum()

id                  0
product             0
component           0
reporter            0
bug_status          0
resolution          0
priority            0
bug_severity        0
version             0
short_desc         58
opendate            0
dup_list        41531
root_id         38016
disc_id         38016
dtype: int64

In [6]:
df = df[['id','short_desc','dup_list']]
df.head()

,id,short_desc,dup_list
0,2,Just testing the Boogzeela setup for log4j,NaN
1,3,file sharing,NaN
2,22,Need hooks for user-defined error handling,NaN
3,27,Error in the formatting of the (%F:%L) layout ...,NaN
4,29,"Calling Category.error(Object, Throwable) thro...",NaN


In [7]:
df.shape

(44049, 3)

Remove Blank Records

In [8]:
blanks = []

for i,id,sd,dl in df.itertuples():
    if type(sd)==str:
        if sd.isspace():
            blanks.append(i)
    else:
        blanks.append(i)
len(blanks)  

58

In [9]:
df.drop(blanks,inplace=True)
df.isnull().sum()

id                0
short_desc        0
dup_list      41474
dtype: int64

**Similarity Methods:**

In [10]:
#conda install -c conda-forge spacy
#conda install -c conda-forge spacy-lookups-data
#!python -m spacy download en_core_web_lg

In [24]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')

In [141]:
def tokenize(df,id):
    s = df[df['id'] == id]
    s = s.short_desc.to_string(index=False)
    s = s[1:]
    # print ('\n'+ s)
    doc = nlp(s)
    #for token in doc:
    #    print(token.text, end=' | ')
    #displacy.render(doc, style='dep', jupyter=True, options={'distance': 110})    
    return doc

In [142]:
doc1 = tokenize(df,103)
doc2 = tokenize(df,104)

**Remove Puntuations**

In [143]:
def show_lemmas(text):
    for token in text:
        print(f'{token.text:{20}} {token.pos_:{6}} {token.lemma:<{22}} {token.lemma_}')

In [144]:
#Remove Puntuations
def remove_puntuation(doc):
    #show_lemmas(doc)
    s = []
    for token in doc:
        if token.is_punct == False:
            s.append(token) 
    #Convert list to string
    str1 = ""    
    for ele in s:  
        str1 += str(ele) +' '     
    return nlp(str1[:-1]) #return doc 

In [145]:
doc1 = remove_puntuation(doc1)

In [146]:
doc2 = remove_puntuation(doc2)

In [147]:
#show_lemmas(doc1)

**Remove Stop Words**

In [148]:
#Remove Stop Words
def remove_stop_word(doc): # accept string
    s = []
    for token in doc:
        if token.is_stop == False:
            s.append(token) 
    #Convert list to string
    str1 = ""    
    for ele in s:  
        str1 += str(ele) +' '     
    return nlp(str1[:-1]) #return doc 

In [149]:
doc1 = remove_stop_word(doc1)
doc1.text

'decoding URL necessary BugRat Repo'

In [150]:
doc2 = remove_stop_word(doc2)
doc2.text

'Add decode description getRequestURI'

In [151]:
print('Similarity: ' + str(doc1.similarity(doc2)))

Similarity: 0.6340470066889127


**Build a function of all previous work**

In [152]:
# Put all prevouse work together in a function
def prep(df,id):
    s = tokenize(df,id)
    s = remove_puntuation(s)
    doc = remove_stop_word(s)
    return doc

In [153]:
# Prepare the vectors for each document
import time
start_time = time.time()
c = 0
prepared = []
l = len(df.index)
for i,id,sd,db in df.itertuples():
    c = c + 1
    doc = prep(df,id)
    prepared.append((id,doc))
    c = c + 1
    if c % 100 == 0:
        progress = c/l*100
        print('Progress: %s %%' % round(progress, 2), end="\r", flush=True)
    if c > 1000:
        print('\nDone')
        break
print("--- %s seconds ---" % (time.time() - start_time))

Progress: 2.27 %
Done
--- 12.281132459640503 seconds ---


In [154]:
c = 0
for i in prepared:
    print(i[0], '\t', i[1].text)
    c = c + 1
    if c > 20:
        break

2 	 testing Boogzeela setup log4j
3 	 file sharing
22 	 Need hooks user defined error handling
27 	 Error formatting F:%L layout
29 	 Calling Category.error(Object Throwable thro
31 	 Conversion log4j Priority types NT Event
32 	 Typo
34 	 StreamCorruptedException SocketAppe
35 	 Custom Priorities lost sent remote
43 	 appletviewer Category bug
44 	 Documentation errors
45 	 Add support CSS properties display
46 	 Add marker support
47 	 Modify thumbnail AOI marker
48 	 hueRotate bug ColorMatrix
49 	 Antialiased clipping
50 	 New layout engine tspan text x y dx dy
51 	 Use double buffer repaint
52 	 Remove ATTR_HREF SVGSyntax
53 	 Implement static LUT Identity Identity
54 	 feOffset dx dy bug


In [160]:
# Calculate one by one similarity score
start_time = time.time()
similarities = []
l = len(prepared)
c = 0
for i in prepared:
    c = c + 1
    for j in prepared:
        score = i[1].similarity(j[1])
        similarities.append((i[0],j[0],score))
    if c % 10 == 0:
        progress = c/l*100
        print('Progress: %s %%' % round(progress, 1), end="\r", flush=True)
print('\nDone')
print("--- %s seconds ---" % (time.time() - start_time))

Progress: 100.0 %
Done
--- 148.55478429794312 seconds ---


In [46]:
dfObj = pd.DataFrame(similarities) 

In [158]:
sorted_similarities = sorted(similarities, key=lambda tup: tup[2], reverse=True)

In [159]:
sorted_similarities[510:550]

[(562, 562, 1.0),
 (563, 563, 1.0),
 (564, 564, 1.0),
 (565, 565, 1.0),
 (566, 566, 1.0),
 (567, 567, 1.0),
 (569, 569, 1.0),
 (355, 475, 0.9999999906303305),
 (475, 355, 0.9999999906303305),
 (48, 441, 0.9999999801943156),
 (441, 48, 0.9999999801943156),
 (48, 373, 0.9999999741066152),
 (48, 553, 0.9999999741066152),
 (373, 48, 0.9999999741066152),
 (373, 441, 0.9999999741066152),
 (441, 373, 0.9999999741066152),
 (441, 553, 0.9999999741066152),
 (553, 48, 0.9999999741066152),
 (553, 441, 0.9999999741066152),
 (373, 553, 0.9999999231865646),
 (553, 373, 0.9999999231865646),
 (317, 341, 0.9673615744389031),
 (341, 317, 0.9673615744389031),
 (144, 145, 0.955040353250707),
 (145, 144, 0.955040353250707),
 (66, 540, 0.9474803672427577),
 (540, 66, 0.9474803672427577),
 (101, 483, 0.9373680380808607),
 (483, 101, 0.9373680380808607),
 (44, 66, 0.9349526529218797),
 (66, 44, 0.9349526529218797),
 (45, 71, 0.9278628880921946),
 (71, 45, 0.9278628880921946),
 (436, 509, 0.9257605126573889),
 

In [77]:
def test(df,id1,id2):
    s1 = to_string(df,id1)
    print('s1: ' + s1)
    s2 = to_string(df,id2)
    print('s2: ' + s2)
    s1 = remove_puntuation(s1)
    s2 = remove_puntuation(s2)
    doc1 = remove_stop_word(s1)
    print('Doc1: ' + doc1.text + '\t')
    doc2 = remove_stop_word(s2)
    print('Doc2: ' + doc2.text + '\t')
    print('Similarity: ' + str(doc1.similarity(doc2)))

In [79]:
test(df,48,441)

s1: hueRotate bug in ColorMatrix
s2: [BUG] BugRat Report#759
Doc1: hueRotate bug ColorMatrix	
Doc2: BUG BugRat Report759	
Similarity: 0.9999999801943156


In [ ]:
dfObj.to_csv('similarities', index = False)